You have to run this notebook sequentially because i overwrite the category id index

In [ ]:
from helpers.db.db_setup_methods import *
from helpers.db.db_query_methods import *

from helpers.db.db_helper_methods import *

#init_db(20000)


The idea here is that typically if you use a wild card like '%charger%' a index on the name would be useless because it cant traverse the index in a manner that will improve speed. It has to do a full scan in order to find every record where it could have a name like '%charger%'. But if you add the name to a index key it would be able to filter out at the index traversal all records where the condition is not satisfied.

So what i expect is that when we have only a index on category_id and not name, it should behave like first finding all the records where it matches the category_id and to this it can use the index. Then it would have to do a full scan on these records to check if the condition applies.

If we then introduce the name to the index then it should be able to find all the records at the first index travelse and dont have to do the check on the clustered traversel. And that should make it quicker

In [ ]:
drop_non_clustered_indexes("products", get_database_name(), output=False)

query = """
SELECT * FROM products WHERE category_id = 1 AND name LIKE '%%charger%%';
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=True)
execute_query("EXPLAIN " + query, database=get_database_name(), print_as_df=True, show_metrics=False)

In [ ]:
drop_non_clustered_indexes("products", get_database_name(), output=False)

query = """
CREATE INDEX idx_products_category_id_and_name ON products(category_id, name);
"""

#execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=False)

query = """
SELECT * FROM products WHERE category_id = 1 AND name LIKE '%%charger%%';
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=True)
execute_query("EXPLAIN " + query, database=get_database_name(), print_as_df=True, show_metrics=False)

What we see is that it is acutally faster. And the difference gets more notable the more data you have. 

Though after reading the use the index luke article https://use-the-index-luke.com/sql/clustering/index-filter-predicates

I did expect the explain to output something different because he uses the orcale database exeuction planner. 

What i expected was first of all more rows in the explain. Like it should first show that it uses the category id index with a type index to get all of the records (or pointers) where the condition is met. And here it should have like a filtered 100.
Then i thought on the second row it would show that now with a ref it will iterate over all of these records and find where the filtered condition is met. 

But this was not the case. Instead i think it is still what happens behind the scenses but mysql just smushes the two into one. 

I did get a bit desapointed after this because i thought there were no difference. But in reality it is show in the extra column. Where the first query has an extra saying using where which indicates that it will iterate over all the records a second time. and the second query has a using index condition indicating that it will filter out everything at the first index traversel. Or atleast i think so